# Global MCMC Fit: JWST + DESI + CMB + Chandra

Ensemble sampler (emcee-inspired) for combined datasets, optimizing alpha for 100% closure.

Prior: Uniform [0,1] for alpha; Gaussian likelihood from chi2 residuals.

In [ ]:
import numpy as np
import emcee
import matplotlib.pyplot as plt
from scipy.stats import norm

# Mock combined residuals (from previous tests: JWST overabundance, DESI H(z), etc.)
# chi2_total = sum over datasets
def log_posterior(alpha):
    # Simplified: chi2 proxy as -sum((data - model(alpha))^2 / sigma^2)
    chi2_jwst = 1.12 * (1 - alpha / 0.2)**2  # From JWST fit
    chi2_desi = 0.68 * (1 + alpha)**2  # DESI
    chi2_cmb = 0.98 * np.exp(alpha)  # CMB
    chi2_chandra = 4.2 * (1 - alpha * 2)**2  # Chandra growth
    chi2_total = chi2_jwst + chi2_desi + chi2_cmb + chi2_chandra
    log_lik = -0.5 * chi2_total
    log_prior = 0 if 0 < alpha < 1 else -np.inf  # Uniform prior
    return log_lik + log_prior

# MCMC setup (emcee)
ndim = 1  # Alpha only
nwalkers = 20
nsteps = 2000
pos = np.random.uniform(0.1, 0.9, (nwalkers, ndim))  # Initial positions
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_posterior)
sampler.run_mcmc(pos, nsteps, progress=True)

# Results
samples = sampler.get_chain(discard=400, flat=True)  # Burn-in 20%
alpha_mean = np.mean(samples)
alpha_std = np.std(samples)
print(f'Alpha posterior: {alpha_mean:.3f} ± {alpha_std:.3f}')

# Plot chain and posterior
fig, ax = plt.subplots(2, 1, figsize=(8, 6))
ax[0].plot(sampler.get_chain()[:, :, 0].T, alpha=0.5)
ax[0].set_title('Alpha Chain')
ax[1].hist(samples, bins=50, density=True)
ax[1].set_title('Posterior')
plt.tight_layout()
plt.savefig('mcmc_alpha.png')
plt.show()